In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
# ------------------------------------------------------------
# Configuration
# ------------------------------------------------------------
DB_NAME = "nba_data.db"
DB_URI = f"sqlite:///{DB_NAME}"
engine = create_engine(DB_URI, echo=False)

In [4]:
# ------------------------------------------------------------
# Load Data & Sort
# ------------------------------------------------------------
df = pd.read_sql("SELECT * FROM player_game_features", engine)

# Ensure data is sorted by player and date
df = df.sort_values(by=["player_id", "game_date"])

# Extract the season or year from 'game_date'
df['game_year'] = pd.to_datetime(df['game_date']).dt.year

# Features and target
features = ["player_id", "pts", "min", "fgm", "fga", "pts_per_min", "fg_pct"]

target = "pts"

df = df.dropna(subset=features + ["pts"])

X = df[features]
y = df["pts"]

In [7]:
# ------------------------------------------------------------
# 2) Helper Function: Sliding-Window Sequences
# ------------------------------------------------------------
def create_player_sequences_sliding_window(data, target, player_column, window_size=20):
    """
    Create sequences of exactly 'window_size' timesteps for each player's timeline.
    Each sequence is the last 'window_size' games leading up to the current game.
    """
    X_list, y_list = [], []

    for p_id, group in data.groupby(player_column):
        # Convert to numpy
        feats = group.drop(columns=[player_column]).values
        targs = target[group.index].values
        
        # Build sequences
        for i in range(len(feats)):
            if i < window_size:
                continue
            seq = feats[i - window_size : i]
            X_list.append(seq)
            y_list.append(targs[i])  # Predict next game (index i)

    X_arr = np.array(X_list, dtype=np.float32)
    y_arr = np.array(y_list, dtype=np.float32)
    return X_arr, y_arr

# ------------------------------------------------------------
# 3) Model Builder
# ------------------------------------------------------------
def build_lstm_model(input_shape):
    """
    A smaller LSTM model (cuDNN-friendly) with dropout and gradient clipping.
    """
    model = Sequential([
        # If zeros are valid data, remove Masking. Otherwise, it can ignore padded zeros.
        Masking(mask_value=0.0, input_shape=input_shape),
        
        # LSTM with default activation=tanh and recurrent_activation=sigmoid for cuDNN
        LSTM(64, return_sequences=False),
        Dropout(0.2),
        
        Dense(16, activation='relu'),
        Dropout(0.2),
        
        Dense(1)  # Regression output
    ])

    # Adam with moderate LR and gradient clipping
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

In [8]:
# ------------------------------------------------------------
# Train on [2015..2022], Validate on 2023
# ------------------------------------------------------------
train_mask = (df['game_year'] >= 2015) & (df['game_year'] <= 2022)
val_mask   = (df['game_year'] == 2023)

train_data = df[train_mask]
val_data   = df[val_mask]

if len(train_data) == 0 or len(val_data) == 0:
    raise ValueError("No data found for given train/validation years.")

# Scale
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train_data[features].drop(columns=["player_id"]))
scaled_val   = scaler.transform(val_data[features].drop(columns=["player_id"]))

train_scaled_df = pd.DataFrame(scaled_train, index=train_data.index, columns=features[1:])
train_scaled_df["player_id"] = train_data["player_id"].values

val_scaled_df = pd.DataFrame(scaled_val, index=val_data.index, columns=features[1:])
val_scaled_df["player_id"] = val_data["player_id"].values

# Build sliding-window sequences
window_size = 20
X_train, y_train = create_player_sequences_sliding_window(
    train_scaled_df, train_data[target], "player_id", window_size=window_size
)
X_val, y_val = create_player_sequences_sliding_window(
    val_scaled_df, val_data[target], "player_id", window_size=window_size
)

# If there's insufficient data, the arrays might be empty
if len(X_train) == 0 or len(X_val) == 0:
    raise ValueError("Not enough data to form sliding-window sequences for train/val.")

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val:   {X_val.shape}, y_val:   {y_val.shape}")

# Convert to tf.data
batch_size = 256
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build & train model
model = build_lstm_model(input_shape=(window_size, X_train.shape[2]))

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Predict on validation
y_pred = model.predict(val_ds).flatten()
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print(f"Validation (2023) MAE:  {mae:.2f}")
print(f"Validation (2023) RMSE: {rmse:.2f}")

X_train: (166396, 20, 6), y_train: (166396,)
X_val:   (5537, 20, 6), y_val:   (5537,)
Epoch 1/50
650/650 [==============================] - 8s 10ms/step - loss: 79.0947 - mae: 6.4881 - val_loss: 47.9221 - val_mae: 5.2502 - lr: 1.0000e-04
Epoch 2/50
650/650 [==============================] - 7s 12ms/step - loss: 41.8313 - mae: 4.9492 - val_loss: 44.8154 - val_mae: 5.1257 - lr: 1.0000e-04
Epoch 3/50
650/650 [==============================] - 7s 11ms/step - loss: 40.6832 - mae: 4.8866 - val_loss: 45.7306 - val_mae: 5.1390 - lr: 1.0000e-04
Epoch 4/50
650/650 [==============================] - 7s 11ms/step - loss: 40.0866 - mae: 4.8579 - val_loss: 44.1781 - val_mae: 5.0662 - lr: 1.0000e-04
Epoch 5/50
650/650 [==============================] - 6s 9ms/step - loss: 39.7117 - mae: 4.8350 - val_loss: 44.6168 - val_mae: 5.0743 - lr: 1.0000e-04
Epoch 6/50
650/650 [==============================] - 6s 9ms/step - loss: 39.7276 - mae: 4.8315 - val_loss: 43.3098 - val_mae: 5.0362 - lr: 1.0000e-04
Epoc